In [18]:
import pandas as pd
import numpy as np
from Bio import AlignIO, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import Counter
from Levenshtein import distance
from copy import deepcopy
from joblib import Parallel, delayed

In [19]:
protease_consensus = SeqIO.read('sequences/hiv-protease-consensus.fasta', 'fasta')
protease_sequences = [s for s in SeqIO.parse('sequences/HIV-1_pol-protease.fasta', 'fasta') if len(s.seq) >= len(protease_consensus.seq)]
rt_consensus = SeqIO.read('sequences/hiv-rt-consensus.fasta', 'fasta')
rt_sequences = [s for s in SeqIO.parse('sequences/HIV-1_pol-RT.fasta', 'fasta') if len(s.seq) >= len(rt_consensus.seq)]

In [20]:
rt_consensus

SeqRecord(seq=Seq('PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGKISKIGPEN...KVL', SingleLetterAlphabet()), id='RT', name='RT', description='RT', dbxrefs=[])

In [21]:
Counter([len(s) for s in protease_sequences])

Counter({99: 23164,
         100: 1185,
         101: 18,
         102: 41,
         103: 148,
         104: 146,
         105: 38,
         106: 23,
         107: 47,
         108: 87,
         109: 138,
         110: 26,
         111: 145,
         112: 19,
         113: 90,
         114: 278,
         115: 229,
         116: 25,
         117: 26,
         118: 40,
         119: 123,
         120: 12,
         121: 29,
         122: 61,
         123: 10,
         124: 12,
         125: 13,
         126: 18,
         127: 27,
         128: 39,
         129: 134,
         130: 25,
         131: 118,
         132: 63,
         133: 41,
         134: 9,
         135: 12,
         136: 7,
         137: 13,
         138: 14,
         139: 12,
         140: 29,
         141: 37,
         142: 43,
         143: 19,
         144: 17,
         145: 29,
         146: 19,
         147: 16,
         148: 13,
         149: 30,
         150: 30,
         151: 288,
         152: 63,
         153: 50

In [ ]:
# Define a function to extract out the protease sequence, even if it's not equal to consensus.

def find_best_match(consensus, query):
    """
    Pass in two SeqRecords. The conversion to strings will be done inside the function.
    """
    query_str = str(query.seq) # <- this is greater than 99 a.a. long.

    min_dist = np.inf
    best_pos = 0
    for pos in range(len(query_str)):
        d_current = distance(str(consensus.seq), str(query_str[pos:pos + len(consensus.seq)]))
        if d_current < min_dist:
            min_dist = d_current
            best_pos = pos
          
    matching_query = SeqRecord(seq=query.seq[best_pos:best_pos+len(consensus.seq)], id=query.id, description=query.description, name=query.name)
    return matching_query

In [ ]:
cleaned_protease_sequences = Parallel(n_jobs=-1)(delayed(find_best_match)(protease_consensus, s) for s in protease_sequences)

In [ ]:
cleaned_rt_sequences = Parallel(n_jobs=-1)(delayed(find_best_match)(rt_consensus, s) for s in rt_sequences)
len(cleaned_rt_sequences)

In [ ]:
SeqIO.write(cleaned_protease_sequences, 'sequences/HIV1-protease.fasta', 'fasta')
SeqIO.write(cleaned_rt_sequences, 'sequences/HIV1-RT.fasta', 'fasta')

In [ ]:
protease_sequences[101236].seq[26:26+99]